<a href="https://colab.research.google.com/github/marybello/RNN-NLP/blob/master/RNN.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [3]:
from keras.datasets import imdb
from keras.preprocessing import sequence
import tensorflow as tf
import os
import numpy as np

Using TensorFlow backend.


In [4]:
vocab_size = 88584
maxlen = 250
batch_size = 64
(train_data, train_labels), (test_data, test_labels) = imdb.load_data(num_words=vocab_size)

17465344/17464789 [==============================] - 0s 0us/step


In [5]:
#because our data are is not equal
train_data = sequence.pad_sequences(train_data, maxlen)
test_data = sequence.pad_sequences(test_data, maxlen)

In [6]:
model= tf.keras.Sequential([
                            tf.keras.layers.Embedding(vocab_size,32),
                            tf.keras.layers.LSTM(32),
                            tf.keras.layers.Dense(1, activation='sigmoid')
                           
                       
])

In [7]:
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding (Embedding)        (None, None, 32)          2834688   
_________________________________________________________________
lstm (LSTM)                  (None, 32)                8320      
_________________________________________________________________
dense (Dense)                (None, 1)                 33        
Total params: 2,843,041
Trainable params: 2,843,041
Non-trainable params: 0
_________________________________________________________________


In [8]:
#compile and train our model

model.compile(
    loss='binary_crossentropy',
    optimizer='rmsprop',
    metrics=['acc']
)
model.fit(train_data, train_labels, epochs=10, validation_split=0.2)

Epoch 1/10
625/625 [==============================] - 39s 62ms/step - loss: 0.4153 - acc: 0.8106 - val_loss: 0.2975 - val_acc: 0.8822
Epoch 2/10
625/625 [==============================] - 38s 60ms/step - loss: 0.2364 - acc: 0.9114 - val_loss: 0.3086 - val_acc: 0.8850
Epoch 3/10
625/625 [==============================] - 38s 60ms/step - loss: 0.1788 - acc: 0.9331 - val_loss: 0.2704 - val_acc: 0.8948
Epoch 4/10
625/625 [==============================] - 38s 60ms/step - loss: 0.1492 - acc: 0.9456 - val_loss: 0.2803 - val_acc: 0.8832
Epoch 5/10
625/625 [==============================] - 37s 60ms/step - loss: 0.1257 - acc: 0.9560 - val_loss: 0.3385 - val_acc: 0.8772
Epoch 6/10
625/625 [==============================] - 37s 60ms/step - loss: 0.1071 - acc: 0.9632 - val_loss: 0.3459 - val_acc: 0.8774
Epoch 7/10
625/625 [==============================] - 38s 60ms/step - loss: 0.0978 - acc: 0.9679 - val_loss: 0.3585 - val_acc: 0.8868
Epoch 8/10
625/625 [==============================] - 38s 60ms

In [9]:
results = model.evaluate(test_data, test_labels)
print(results)

782/782 [==============================] - 12s 16ms/step - loss: 0.5984 - acc: 0.8429
[0.5983681082725525, 0.8429200053215027]


In [12]:
from tensorflow import keras
word_index = imdb.get_word_index()
def encode_text(text):
  tokens = keras.preprocessing.text.text_to_word_sequence(text)
  tokens = [word_index[word] if word in word_index else 0 for word in tokens]
  return sequence.pad_sequences([tokens],maxlen)[0]
text = 'that movie was just so amazing, so amazing'
encoded = encode_text(text)
encoded  




array([  0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,
         0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,
         0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,
         0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,
         0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,
         0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,
         0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,
         0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,
         0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,
         0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,
         0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,
         0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,
         0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,
         0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   

In [13]:
reverse_word_index = {value: key for (key,value) in word_index.items()}
 
def decode_integer(integers):
  pad = 0
  text =""
  for num in integers:
    if num != pad:
      text += reverse_word_index[num] + ' '
  return text[:-1]    
print(decode_integer(encoded))

that movie was just so amazing so amazing


In [18]:
def predict(text):
  encoded_text = encode_text(text)
  pred = np.zeros((1,250))
  pred[0]  = encoded_text
  result = model.predict(pred)
  print(result[0])
positive_review ="The movie was totally awesome, i really loved it and i would watch it again beacue it was great"
predict(positive_review)  
negative_review = "The movie really sucked, i hated it and i would not watch it again because it's one of the worst movies i've seen"
predict(negative_review)

[0.53148574]
[0.5001014]


In [19]:
#creating a model that is capable of predicting the next character in a  sequence
from keras.preprocessing import sequence
import keras
import os
import tensorflow as tf
import numpy as np


In [20]:
path_to_file = keras.utils.get_file('shakespear.txt', 'https://storage.googleapis.com/download.tensorflow.org/data/shakespeare.txt')

1122304/1115394 [==============================] - 0s 0us/step


In [24]:
#to select your own file from your computer
#from google.colab import files
#path_to_file = list(files.upload().keys())[0]

In [23]:
text = open(path_to_file, 'rb').read().decode(encoding='utf-8')
print('length of text is {} characters'.format(len(text)))

length of text is 1115394 characters


In [25]:
text[:250]

'First Citizen:\nBefore we proceed any further, hear me speak.\n\nAll:\nSpeak, speak.\n\nFirst Citizen:\nYou are all resolved rather to die than to famish?\n\nAll:\nResolved. resolved.\n\nFirst Citizen:\nFirst, you know Caius Marcius is chief enemy to the people.\n'

In [29]:
vocab = sorted(set(text))
#creating a mapping from unique characters to indices
char2idx = {u:i for i,u in enumerate(vocab)}
idx2char = np.array(vocab)
def text_to_int(text):
  return np.array([char2idx[c] for c in text])
text_as_int = text_to_int(text)
print("Text", text[:13])
text_to_int(text[:13])

Text First Citizen


array([18, 47, 56, 57, 58,  1, 15, 47, 58, 47, 64, 43, 52])

In [33]:
#writing an opposite function
def int_to_text(ints):
  try:
    ints = ints.numpy()
  except:
    pass
  return ''.join(idx2char[ints])
print(int_to_text([6,53,54,23,4,5,1,45,22,3,26,26]))      

,opK&' gJ$NN


In [34]:
seq_length = 100
examples_per_epoch = len(text)//(seq_length+1)
#Create training examples
char_dataset = tf.data.Dataset.from_tensor_slices(text_as_int)

In [35]:
sequences= char_dataset.batch(seq_length+1, drop_remainder=True)

In [36]:
def split_input_target(chunk):
  input_text = chunk[:-1]
  target_text = chunk[1:]
  return input_text, target_text
dataset = sequences.map(split_input_target)  

In [37]:
for x,y in dataset.take(2):
  print('\n\nExample\n')
  print('INPUT')
  print(int_to_text(x))
  print('\n OUTPUT')
  print(int_to_text(y))



Example

INPUT
First Citizen:
Before we proceed any further, hear me speak.

All:
Speak, speak.

First Citizen:
You

 OUTPUT
irst Citizen:
Before we proceed any further, hear me speak.

All:
Speak, speak.

First Citizen:
You 


Example

INPUT
are all resolved rather to die than to famish?

All:
Resolved. resolved.

First Citizen:
First, you 

 OUTPUT
re all resolved rather to die than to famish?

All:
Resolved. resolved.

First Citizen:
First, you k


In [38]:
batch_size = 64
vocab_size = len(vocab)
embedding = 256
rnn_units = 1024
buffer_size =10000
data = dataset.shuffle(buffer_size).batch(batch_size, drop_remainder=True)

In [40]:
def build_model(vocab_size,embedding,rnn_units, batch_size):
  model = tf.keras.Sequential([
                               tf.keras.layers.Embedding(vocab_size,embedding, batch_input_shape=[batch_size,None]),
                               tf.keras.layers.LSTM(rnn_units, return_sequences=True, stateful=True,recurrent_initializer='glorot_uniform'),
                               tf.keras.layers.Dense(vocab_size)
  ])
  return model
model = build_model(vocab_size,embedding,rnn_units, batch_size)
model.summary()  

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_2 (Embedding)      (64, None, 256)           16640     
_________________________________________________________________
lstm_1 (LSTM)                (64, None, 1024)          5246976   
_________________________________________________________________
dense_1 (Dense)              (64, None, 65)            66625     
Total params: 5,330,241
Trainable params: 5,330,241
Non-trainable params: 0
_________________________________________________________________


In [41]:
for input_example_batch, target_example_batch in data.take(1):
  example_batch_predictions = model(input_example_batch)
  print(example_batch_predictions.shape)

(64, 100, 65)


In [42]:
def loss(labels, logits):
  return tf.keras.losses.sparse_categorical_crossentropy(labels, logits, from_logits=True)

In [44]:
model.compile(optimizer='adam',loss=loss)

In [45]:
checkpoint_dir = './training_checkpoints'
checkpoint_prefix = os.path.join(checkpoint_dir,'ckpt_{epoch}')
checkpoint_callback= tf.keras.callbacks.ModelCheckpoint(
    filepath=checkpoint_prefix,
    save_weights_only=True
)

In [ ]:
history = model.fit(data,epochs=40,callbacks=[checkpoint_callback])


Epoch 1/40
172/172 [==============================] - 28s 165ms/step - loss: 2.7589
Epoch 2/40
172/172 [==============================] - 28s 165ms/step - loss: 2.0972
Epoch 3/40
172/172 [==============================] - 28s 165ms/step - loss: 1.8426
Epoch 4/40
172/172 [==============================] - 28s 165ms/step - loss: 1.6724
Epoch 5/40
172/172 [==============================] - 28s 165ms/step - loss: 1.5555
Epoch 6/40
172/172 [==============================] - 28s 165ms/step - loss: 1.4768
Epoch 7/40
172/172 [==============================] - 28s 165ms/step - loss: 1.4191
Epoch 8/40
172/172 [==============================] - 28s 165ms/step - loss: 1.3743
Epoch 9/40
172/172 [==============================] - 28s 165ms/step - loss: 1.3368
Epoch 10/40
172/172 [==============================] - 28s 165ms/step - loss: 1.3037
Epoch 11/40
172/172 [==============================] - 28s 165ms/step - loss: 1.2725
Epoch 12/40
172/172 [==============================] - 28s 165ms/step - lo

In [ ]:
model = build_model(vocab_size,embedding,rnn_units,batch_size=1)


In [ ]:
model.load_weights(tf.train_latest_checkpoint(checkpoint_dir))
model.build(tf.TensorShape([1,None]))